In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from types import SimpleNamespace
from sklearn.preprocessing import MinMaxScaler
import os

In [20]:
# config 설정에서 타겟 크기에 따라 output_size 수정
config = {
    "learning_rate": 0.0001,
    "epoch": 1,
    "batch_size": 64,
    "hidden_size": 64,
    "num_layers": 2,
    "output_size": 3  # 타겟과 맞추기
}
CFG = SimpleNamespace(**config)

품목_리스트 = ['건고추', '사과', '감자', '배', '깐마늘(국산)', '무', '상추', '배추', '양파', '대파']

In [21]:
class AgriculturePriceDataset(Dataset):
    def __init__(self, dataframe, window_size=9, prediction_length=3, is_test=False):
        self.data = dataframe
        self.window_size = window_size
        self.prediction_length = prediction_length
        self.is_test = is_test
        
        self.price_column = '평균가격(원)'
        self.numeric_columns = self.data.select_dtypes(include=[np.number]).columns.tolist()
        
        self.sequences = []
        if not self.is_test:
            for i in range(len(self.data) - self.window_size - self.prediction_length + 1):
                x = self.data[self.numeric_columns].iloc[i:i+self.window_size].values
                y = self.data[self.price_column].iloc[i+self.window_size:i+self.window_size+self.prediction_length].values
                self.sequences.append((x, y))
        else:
            self.sequences = [self.data[self.numeric_columns].values]
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        if not self.is_test:
            x, y = self.sequences[idx]
            return torch.FloatTensor(x), torch.FloatTensor(y)
        else:
            return torch.FloatTensor(self.sequences[idx])

In [22]:

class PricePredictionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PricePredictionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        batch_y = batch_y[..., :CFG.output_size]  # target의 마지막 차원을 output_size에 맞춰줌
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)  

def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
    return total_loss / len(test_loader)

In [23]:
# 최종 프로세스 함수
def process_data(raw_file, 산지공판장_file, 전국도매_file, 품목명, scaler=None):
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler

    # 데이터 로드
    raw_data = pd.read_csv(raw_file)
    산지공판장 = pd.read_csv(산지공판장_file)
    전국도매 = pd.read_csv(전국도매_file)

    # 품목별 조건 설정
    conditions = {
        '감자': {'weight': 20,
        'target': lambda df: (df['품종명'] == '감자 수미') & (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['감자'], '품종명': ['수미'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['감자'], '품종명': ['수미']}
    },
    '건고추': {'weight': 1,
        'target': lambda df: (df['품종명'] == '화건') & (df['거래단위'] == '30 kg') & (df['등급'] == '상품'),
        '공판장': None, 
        '도매': None  
    },
    '깐마늘(국산)': {'weight': 20,
        'target': lambda df: (df['거래단위'] == '20 kg') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['마늘'], '품종명': ['깐마늘'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['마늘'], '품종명': ['깐마늘']}
    },
    '대파': {'weight': 1,
        'target': lambda df: (df['품종명'] == '대파(일반)') & (df['거래단위'] == '1키로단') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['대파'], '품종명': ['대파(일반)'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['대파'], '품종명': ['대파(일반)']}
    },
    '무': {'weight': 20,
        'target': lambda df: (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['무'], '품종명': ['기타무'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['무'], '품종명': ['무']}
    },
    '배추': {'weight': 10,
        'target': lambda df: (df['거래단위'] == '10키로망대') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배추'], '품종명': ['쌈배추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배추'], '품종명': ['배추']}
    },
    '사과': {'weight': 2,
        'target': lambda df: (df['품종명'].isin(['홍로', '후지'])) & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['사과'], '품종명': ['후지'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['사과'], '품종명': ['후지']}
    },
    '상추': {'weight': 0.1,
        'target': lambda df: (df['품종명'] == '청') & (df['거래단위'] == '100 g') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['상추'], '품종명': ['청상추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['상추'], '품종명': ['청상추']}
    },
    '양파': {'weight': 1,
        'target': lambda df: (df['품종명'] == '양파') & (df['거래단위'] == '1키로') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['양파'], '품종명': ['기타양파'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['양파'], '품종명': ['양파(일반)']}
    },
    '배': {'weight': 5,
        'target': lambda df: (df['품종명'] == '신고') & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배'], '품종명': ['신고'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배'], '품종명': ['신고']}
    }
    }
    
    # 해당 품목 데이터 필터링
    raw_품목 = raw_data[raw_data['품목명'] == 품목명]
    target_mask = conditions[품목명]['target'](raw_품목)
    filtered_data = raw_품목[target_mask].copy()

    # '산지공판장' 또는 '전국도매'에서 필요한 열 추출
    filtered_산지공판장 = 산지공판장[산지공판장['품목명'] == 품목명]
    filtered_도매 = 전국도매[전국도매['품목명'] == 품목명]

    # 총거래금액과 총반입량을 기준으로 1kg 거래금액 계산
    filtered_산지공판장['1kg_거래금액'] = filtered_산지공판장['총거래금액(원)'] / filtered_산지공판장['총반입량(kg)']
    filtered_산지공판장['가중_1kg_거래금액'] = filtered_산지공판장['1kg_거래금액'] * conditions[품목명]['weight']

    # '시점' 기준으로 raw_data에 병합
    filtered_data = filtered_data.merge(
        filtered_산지공판장[['시점', '1kg_거래금액', '가중_1kg_거래금액']],
        on='시점',
        how='left'
    )

    # 공판장 및 도매 데이터 처리 후 병합
    if conditions[품목명]['공판장']:
        filtered_공판장 = 산지공판장
        for key, value in conditions[품목명]['공판장'].items():
            filtered_공판장 = filtered_공판장[filtered_공판장[key].isin(value)]
        
        filtered_공판장 = filtered_공판장.add_prefix('공판장_').rename(columns={'공판장_시점': '시점'})
        filtered_data = filtered_data.merge(filtered_공판장, on='시점', how='left')

    if conditions[품목명]['도매']:
        filtered_도매 = 전국도매
        for key, value in conditions[품목명]['도매'].items():
            filtered_도매 = filtered_도매[filtered_도매[key].isin(value)]
        
        filtered_도매 = filtered_도매.add_prefix('도매_').rename(columns={'도매_시점': '시점'})
        filtered_data = filtered_data.merge(filtered_도매, on='시점', how='left')

    # '시점'별 평균가격(원)의 표준편차 계산 후 추가
    std_df = filtered_data.groupby('시점')['평균가격(원)'].std().reset_index().rename(columns={'평균가격(원)': '품목별_표준편차'})
    filtered_data = filtered_data.merge(std_df, on='시점', how='left')

    # 수치형 컬럼 처리
    numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns
    filtered_data = filtered_data[['시점'] + list(numeric_columns)]
    filtered_data[numeric_columns] = filtered_data[numeric_columns].replace([np.inf, -np.inf], np.nan)  # infinity 값을 NaN으로 변환
    filtered_data[numeric_columns] = filtered_data[numeric_columns].fillna(0)  # NaN 값을 0으로 대체

    # 정규화 적용
    if scaler is None:
        scaler = MinMaxScaler()
        filtered_data[numeric_columns] = scaler.fit_transform(filtered_data[numeric_columns])
    else:
        filtered_data[numeric_columns] = scaler.transform(filtered_data[numeric_columns])
    return filtered_data, scaler

In [24]:
import os
import torch
import numpy as np
import mlflow
import mlflow.pytorch
from tqdm import tqdm
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

# MLflow 설정
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("container_farmer3")

# 품목별 예측과 스케일러 저장
품목별_predictions = {}
품목별_scalers = {}

pbar_outer = tqdm(품목_리스트, desc="품목 처리 중", position=0)

for 품목명 in pbar_outer:
    pbar_outer.set_description(f"품목별 전처리 및 모델 학습 -> {품목명}")
    
    train_data, scaler = process_data(
        "C:/Users/medici/pybasic/train.csv", 
        "C:/Users/medici/pybasic/TRAIN_산지공판장_2018-2021.csv", 
        "C:/Users/medici/pybasic/TRAIN_전국도매_2018-2021.csv", 
        품목명
    )
    
    품목별_scalers[품목명] = scaler
    dataset = AgriculturePriceDataset(train_data)

    # Dataset 분할
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    # DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)

    input_size = len(dataset.numeric_columns)
    model = PricePredictionLSTM(input_size, CFG.hidden_size, CFG.num_layers, CFG.output_size)

    criterion = torch.nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), CFG.learning_rate)
    best_val_loss = float('inf')
    os.makedirs('models', exist_ok=True)

    # MLflow 실험 시작
    with mlflow.start_run(run_name=f"{품목명}_prediction"):
        mlflow.log_param("품목명", 품목명)

        for epoch in range(CFG.epoch):
            train_loss = train_model(model, train_loader, criterion, optimizer, CFG.epoch)
            val_loss = evaluate_model(model, val_loader, criterion)

            # 학습과 검증 메트릭 로깅
            mlflow.log_metrics({"train_loss": train_loss, "val_loss": val_loss})

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), f'models/best_model_{품목명}.pth')
                mlflow.pytorch.log_model(model, artifact_path=f"model_{품목명}")

            print(f'Epoch {epoch+1}/{CFG.epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        print(f'Best Validation Loss for {품목명}: {best_val_loss:.4f}')

        # 테스트 데이터에 대해 추론 수행 및 로깅
        품목_predictions = []
        test_losses = []
        pbar_inner = tqdm(range(25), desc="테스트 파일 추론 중", position=1, leave=False)
        
        for i in pbar_inner:
            test_file = f"C:/Users/medici/pybasic/test/TEST_{i:02d}.csv"
            산지공판장_file = f"C:/Users/medici/pybasic/test/meta/TEST_산지공판장_{i:02d}.csv"
            전국도매_file = f"C:/Users/medici/pybasic/test/meta/TEST_전국도매_{i:02d}.csv"

            test_data, _ = process_data(
                test_file, 산지공판장_file, 전국도매_file, 품목명, scaler=품목별_scalers[품목명]
            )
            test_dataset = AgriculturePriceDataset(test_data, is_test=True)
            test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

            model.eval()
            predictions = []
            test_loss_sum = 0
            with torch.no_grad():
                for batch in test_loader:
                    output = model(batch)
                    batch_y = batch[..., :CFG.output_size]  # `output_size`에 맞추어 타겟 차원 조정
                    predictions.append(output.numpy())
                    test_loss_sum += criterion(output, batch_y).item()  # 손실 계산 시 맞춘 차원 사용

            # 평균 손실 계산 및 누적
            test_loss_avg = test_loss_sum / len(test_loader)
            test_losses.append(test_loss_avg)

            predictions_array = np.concatenate(predictions)

            # 예측값 복원
            price_column_index = test_data.columns.get_loc(test_dataset.price_column)
            predictions_reshaped = predictions_array.reshape(-1, 1)

            price_scaler = MinMaxScaler()
            price_scaler.min_ = 품목별_scalers[품목명].min_[price_column_index]
            price_scaler.scale_ = 품목별_scalers[품목명].scale_[price_column_index]
            predictions_original_scale = price_scaler.inverse_transform(predictions_reshaped)

            if np.isnan(predictions_original_scale).any():
                pbar_inner.set_postfix({"상태": "NaN"})
            else:
                pbar_inner.set_postfix({"상태": "정상"})
                품목_predictions.extend(predictions_original_scale.flatten())

        # 테스트 파일별 평균 손실과 전체 손실 로깅
        품목별_predictions[품목명] = 품목_predictions
        avg_test_loss = np.mean(test_losses)
        mlflow.log_metric("avg_test_loss", avg_test_loss)
        print(f'Average Test Loss for {품목명}: {avg_test_loss:.4f}')

        # 예측 결과 아티팩트로 저장
        mlflow.log_artifact(f'models/best_model_{품목명}.pth')

    pbar_outer.update(1)


품목별 전처리 및 모델 학습 -> 건고추:   0%|          | 0/10 [00:00<?, ?it/s]

2024/10/31 17:44:11 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.0+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.5.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/10/31 17:44:15 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.0+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.5.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/10/31 17:44:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 1/1, Train Loss: 0.4338, Val Loss: 0.4309
Best Validation Loss for 건고추: 0.4309


c:\Users\medici\miniconda3\envs\syrup\lib\site-packages\torch\nn\modules\loss.py:128: UserWarning: Using a target size (torch.Size([1, 9, 3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
c:\Users\medici\miniconda3\envs\syrup\lib\site-packages\torch\nn\modules\loss.py:128: UserWarning: Using a target size (torch.Size([1, 9, 3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
c:\Users\medici\miniconda3\envs\syrup\lib\site-packages\torch\nn\modules\loss.py:128: UserWarning: Using a target size (torch.Size([1, 9, 3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please 

Average Test Loss for 건고추: 0.4306


...

Average Test Loss for 양파: 0.2869


C:\Users\medici\AppData\Local\Temp\ipykernel_36392\1708597175.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_산지공판장['1kg_거래금액'] = filtered_산지공판장['총거래금액(원)'] / filtered_산지공판장['총반입량(kg)']
C:\Users\medici\AppData\Local\Temp\ipykernel_36392\1708597175.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_산지공판장['가중_1kg_거래금액'] = filtered_산지공판장['1kg_거래금액'] * conditions[품목명]['weight']
2024/10/31 17:50:32 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.0+cu118) contains a loca

Epoch 1/1, Train Loss: 0.1600, Val Loss: 0.1526
Best Validation Loss for 대파: 0.1526


C:\Users\medici\AppData\Local\Temp\ipykernel_36392\1708597175.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_산지공판장['1kg_거래금액'] = filtered_산지공판장['총거래금액(원)'] / filtered_산지공판장['총반입량(kg)']
C:\Users\medici\AppData\Local\Temp\ipykernel_36392\1708597175.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_산지공판장['가중_1kg_거래금액'] = filtered_산지공판장['1kg_거래금액'] * conditions[품목명]['weight']
c:\Users\medici\miniconda3\envs\syrup\lib\site-packages\torch\nn\modules\loss.py:128: UserWarning: Using a tar

Average Test Loss for 대파: 0.1810


In [25]:
sample_submission = pd.read_csv("C:/Users/medici/pybasic/sample_submission.csv")

for 품목명, predictions in 품목별_predictions.items():
    sample_submission[품목명] = predictions

# 결과 저장
sample_submission.to_csv('C:/Users/medici/pybasic/baseline_submission7.csv', index=False)